In [1]:
!pip install -qU langchain_community pypdf langchain-groq sentence_transformers faiss-gpu langgraph pydantic gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
# LIBRARY
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"
os.environ['TAVILY_API_KEY'] = "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"

In [4]:
# UPLOAD THE DOCUMENTS
documents = PyPDFLoader('/kaggle/input/cooking-dataset/cooking_book.pdf')
documents = documents.load() 

In [5]:
# SPLIT THE DOCUMENTS INTO CHUNK
txt_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150,length_function=len)
splits = txt_spliter.split_documents(documents)
print(f"split the documents into {len(splits)} chunks")

split the documents into 128 chunks


In [6]:
#Create embedding
model_name = 'BAAI/bge-small-en'
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}

In [7]:
HF_embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

<ipython-input-7-801c86d419aa>:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HF_embedding = HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#Vector store 
vectorstore = FAISS.from_documents(documents=splits, embedding=HF_embedding)
retriever = vectorstore.as_retriever()

In [9]:
def docs2str(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [10]:
# RAG
template = """Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

final_rag_chain = ({"context": retriever | docs2str, "question": RunnablePassthrough()} 
                   | prompt
                   | llm
                   | StrOutputParser()
)

In [11]:
question = "how to cook Haleem?"
response = final_rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: how to cook Haleem?
Answer: To cook Haleem, follow this recipe:

1. Soak rice, wheat, and dals (channa dal, mash dal, masoor dal) overnight.
2. Mix haldi (turmeric), chilli powder, salt, onions, ginger paste, and 2 glasses of water with beef and cook till beef is tender.
3. Fry the beef in 50 ml of oil. Remove beef pieces from curry.
4. In the same oil, fry onions until golden brown. Add garam masala powder and curry masala to the fried onions.
5. Blend the soaked rice, wheat, and dals into a smooth paste.
6. Add the paste to the fried onions and stir. Add the beef pieces, remaining water, and cook until the water dries.
7. For the filling, mix boiled eggs, coriander leaves, mint leaves, green chilies, and chat masala.
8. Form the mixture into kababs and shallow fry until crisp and golden brown.
9. Serve hot with chutney.

Note: The recipe seems to have a mix of two different dishes - Haleem and Kebabs. The Haleem is a stew-like dish made with meat, lentils, and spices, while

**---------------------------------------------------------------------------The more advance RAG System---------------------------------------------------------------------------------**

In [12]:
#
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "How to cook Haleem?", "chat_history": []}))

You're asking for instructions on how to prepare Haleem. Here's the standalone question: "What are the steps to cook Haleem?"


In [13]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Ingradient for cooking Haleem?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "how to cook Haleem?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Ingradient for cooking Haleem?
AI: The ingredients for cooking Haleem are:
1 kg boneless beef
2 tablespoons ginger garlic paste
3 medium onions
2 tablespoons chilli powder
1 teaspoon haldi (turmeric)
300 ml oil
2 tablespoons salt
2 tablespoons garam masala powder
half cup channa dal (split chickpeas)
half cup mash dal (split green gram)
half cup masoor dal (red lentils)
half cup rice
1 cup wheat
For the filling:
4 boiled eggs, finely chopped
1 cup coriander leaves, finely chopped
1 cup mint leaves, finely chopped
2 or 3 green chilies, finely chopped
1 teaspoon chat masala
For frying:
1/4 cup besan (gram flour)
2 eggs
Additionally, for the initial soaking:
haldi, chilli, salt, onions, ginger paste, and 2 glasses water
For cooking the meat:
black pepper, cloves, bari elaichi (black cardamom), jeera (cumin), cinnamon, kari patta (curry leaves), and 3-4 glasses of water.

Note: Some of the ingredients are repeated in the method section, I have included them for clarity.

Human: how 

**-------------------------------------Gradio----------------------------------**

In [16]:
import gradio as gr

def generate_answer(messages, state):
    user_message = messages[-1]
    chat_history = [{'role': 'user', 'content': msg} for msg in messages]
    input_dict = {"input": user_message, "chat_history": chat_history}
    response = rag_chain.invoke(input_dict)
    return response['answer']

In [22]:
import gradio as gr

gr.ChatInterface(
    fn=generate_answer, 
    type="messages",
    textbox=gr.Textbox(placeholder="Type your cooking-related question here...", container=False, scale=7),
    title="Smart Kitchen Assistant",
    chatbot=gr.Chatbot(height=300, type="messages"),
    description="Welcome to the Smart Kitchen Assistant! Powered by Retrieval-Augmented Generation (RAG), "
        "this tool provides detailed cooking instructions. Simply ask about any recipe, and get "
        "a list of required ingredients followed by step-by-step instructions to prepare the dish.",
).launch()

* Running on local URL:  http://127.0.0.1:7865
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://91d1cb445dfae77742.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
